In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Autumn_2025/42028_Deep_Learning/Assignments/assignment 3/code

/content/drive/MyDrive/Autumn_2025/42028_Deep_Learning/Assignments/assignment 3/code


In [3]:
ls

'Data Downloader.ipynb'  'Prevous yolo runs'/   yolo11n.pt
 datasets/                runs_bad/            'yolo12m (1).pt'
 drone/                   runs_bad2/            yolo12m.pt
 face_data/               runs_bad3/            yolo12n.pt
'facial recognition'/     runs_old_goog/        Yolov5_StrongSORT_OSNet/
 human+face/              yolo/
 keras/                  'yolo11n (1).pt'


In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

  from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Wed May 14 07:18:45 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             44W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

# ⚙️ 1. Setup paths and parameters


In [5]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 112.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [6]:
import torch
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("GPU memory cleared.")
else:
    print("No GPU available.")

GPU memory cleared.


In [7]:
import os
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [8]:
database = "/content/drive/MyDrive/Autumn_2025/42028_Deep_Learning/Assignments/assignment 3/code/datasets/mix dataset"


In [9]:
data_yaml_path = f"{database}/data.yaml"


In [10]:
data_yaml_path

'/content/drive/MyDrive/Autumn_2025/42028_Deep_Learning/Assignments/assignment 3/code/datasets/mix dataset/data.yaml'

In [11]:
# Load a model
model = YOLO("yolo12n.yaml")  # build a new model from YAML
model = YOLO("yolo12n.pt")  # load a pretrained model (recommended for training)
model = YOLO("yolo12n.yaml").load("yolo12n.pt")  # build from YAML and transfer weights

Transferred 691/691 items from pretrained weights


In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [13]:
# Train the model
results = model.train(
    data=data_yaml_path,
    epochs=500,
    imgsz=640,
    batch=128,
    workers=8,
    pretrained=True,
    cache='ram',
    device= device,
    optimizer='AdamW',
    lr0=0.001,
    cos_lr=True,
    warmup_epochs=3,
    lrf=0.01,
    weight_decay=0.0005,
    momentum=0.937,
    project='runs/yolov12n',
    name='train',
    exist_ok=True,
    save_period = 50
)



engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=128, bgr=0.0, box=7.5, cache=ram, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/Autumn_2025/42028_Deep_Learning/Assignments/assignment 3/code/datasets/mix dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=500, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12n.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pret

100%|██████████| 755k/755k [00:00<00:00, 14.0MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  2    180864  ultralytics.nn.modules.block.A2C2f           [128, 128, 2, True, 4]        
  7                  -1  1    295424  ultralytics

YOLO12n summary: 272 layers, 2,568,243 parameters, 2,568,227 gradients, 6.5 GFLOPs

Transferred 640/691 items from pretrained weights
Freezing layer 'model.21.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.5±0.1 ms, read: 0.1±0.1 MB/s, size: 61.7 KB)


train: Scanning /content/drive/MyDrive/Autumn_2025/42028_Deep_Learning/Assignments/assignment 3/code/datasets/mix dataset/train/labels.cache... 3228 images, 6 backgrounds, 0 corrupt: 100%|██████████| 3234/3234 [00:00<?, ?it/s]


...

    389/500      25.2G     0.5657     0.3822     0.9649         65        640: 100%|██████████| 26/26 [00:13<00:00,  1.98it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.54it/s]

                   all        165        186      0.909      0.915      0.955      0.773
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 289, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



389 epochs completed in 1.565 hours.
Optimizer stripped from runs/yolov12n/train/weights/last.pt, 5.6MB
Optimizer stripped from runs/yolov12n/train/weights/best.pt, 5.6MB

Validating runs/yolov12n/train/weights/best.pt...
Ultralytics 8.3.133 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO12n summary (fused): 159 layers, 2,556,923 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.37it/s]


                   all        165        186      0.934      0.919      0.964      0.793
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/yolov12n/train


# 7. 📊Visualization the results

In [14]:
# Load your trained model
model = YOLO("runs/yolov12n/train/weights/best.pt")

# Evaluate on the validation set defined in your data.yaml
metrics = model.val()




Ultralytics 8.3.133 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO12n summary (fused): 159 layers, 2,556,923 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 10.0±21.0 ms, read: 18.4±16.6 MB/s, size: 43.6 KB)


val: Scanning /content/drive/MyDrive/Autumn_2025/42028_Deep_Learning/Assignments/assignment 3/code/datasets/mix dataset/valid/labels.cache... 165 images, 0 backgrounds, 0 corrupt: 100%|██████████| 165/165 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:02<00:00,  4.58it/s]


                   all        165        186      0.933      0.919      0.964      0.794
Speed: 1.4ms preprocess, 3.2ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs/detect/val


In [15]:
# Mean metrics across all classes
print(f"mAP@0.5: {metrics.box.map50:.3f}")
print(f"mAP@0.5:0.95: {metrics.box.map:.3f}")
print(f"Mean Precision: {metrics.box.mp:.3f}")
print(f"Mean Recall: {metrics.box.mr:.3f}")



mAP@0.5: 0.964
mAP@0.5:0.95: 0.794
Mean Precision: 0.933
Mean Recall: 0.919


In [16]:
p, r, ap50, ap = metrics.box.class_result(0)
print(f"{} — Precision: {p:.3f}, Recall: {r:.3f}, AP@0.5: {ap50:.3f}, AP@0.5:0.95: {ap:.3f}")


NameError: name 'name' is not defined

# continue training later from the last checkpoint

In [ ]:
# # Load a model
# model = YOLO("runs/yolov12m-human/train/weights/epoch.pt")  # load a partially trained model

# # Resume training
# results = model.train(
#     epochs=4,
#     imgsz=640,
#     batch=32,
#     workers=8,
#     pretrained=True,
#     cache='ram',
#     device='cuda:0',
#     optimizer='AdamW',
#     lr0=0.001,
#     cos_lr=True,
#     warmup_epochs=3,
#     lrf=0.01,
#     weight_decay=0.0005,
#     momentum=0.937,
#     project='runs/yolov12m-human',
#     name='continue train',
#     exist_ok=True,
#     save_period = 1,
#     resume=True
# )

In [ ]:

# # Load your trained model
# model = YOLO("runs/yolov12m-human/train/weights/epoch3.pt")

# # Evaluate on the validation set defined in your data.yaml
# metrics = model.val()




In [ ]:
# # Mean metrics across all classes
# print(f"mAP@0.5: {metrics.box.map50:.3f}")
# print(f"mAP@0.5:0.95: {metrics.box.map:.3f}")
# print(f"Mean Precision: {metrics.box.mp:.3f}")
# print(f"Mean Recall: {metrics.box.mr:.3f}")



In [ ]:
#p, r, ap50, ap = metrics.box.class_result(0)
#print(f"{name} — Precision: {p:.3f}, Recall: {r:.3f}, AP@0.5: {ap50:.3f}, AP@0.5:0.95: {ap:.3f}")

